# Mosquito Project - Spatiotemporal Analysis 
This study explores the spatiotemporal patterns of mosquito surveillance data by conducting both spatial and temporal analyses. Using gridded spatial mapping and aggregated time-based plots, the aim is to characterize mosquito abundance, species distribution, sampling efforts, disease positivity rates, and pathogen compositions across the study area and periods of interest. Spatial and temporal patterns provide complementary perspectives that together offer a comprehensive understanding of how mosquito populations and disease risks vary across both geography and time.

## Outlier Detection and Preprocessing
To ensure data quality before conducting spatiotemporal analysis, we applied a two-step outlier detection process to the mosquito count data. First, the study area was divided into 1-mile by 1-mile spatial grid cells, which will also be used in the spatial analysis in the following sections. This gridding helps localize trends and supports consistent comparisons across areas. The data was also grouped into 3-month time periods (quarterly) to account for seasonal variation and reduce the impact of long-term trends or short-term noise.

To normalize the right-skewed distribution of mosquito counts and stabilize variance, we applied a log transformation using log(x + 1) to each count. Then, within each grid cell and time period, we used the Interquartile Range (IQR) method to identify and remove unusually high values. Specifically, any log-transformed value greater than the 75th percentile (Q3) plus 5 times the IQR was classified as an outlier. This higher threshold (compared to the conventional 1.5×IQR) was chosen to avoid excluding rare but biologically meaningful spikes in mosquito counts, such as those that might occur during localized outbreaks.

In addition to statistical filtering, we applied a second rule to remove any count values that ended in two or more zeros (e.g., 100, 300, 1000). These values are often indicative of manual rounding, entry errors, or placeholder estimates, and may not reflect actual observations. This rule helped identify suspicious entries that might not be flagged by statistical methods alone.

Together, these steps reduced the influence of data anomalies while preserving meaningful variation, ensuring that the subsequent spatiotemporal analysis would be based on clean and reliable data.

In [ ]:
import pandas as pd
import numpy as np
from math import ceil, cos, radians

# Load Data 
data = pd.read_csv("Clean_Data (keeping potential outliers).csv")

# Setup Grid (1 mile x 1 mile) 
cell_size = 1.0  # miles
min_lon, max_lon = data['Longitude'].min(), data['Longitude'].max()
min_lat, max_lat = data['Latitude'].min(), data['Latitude'].max()

miles_per_deg_lat = 69.0
center_lat = (min_lat + max_lat) / 2
miles_per_deg_lon = 69.0 * cos(radians(center_lat))

lat_span_miles = (max_lat - min_lat) * miles_per_deg_lat
lon_span_miles = (max_lon - min_lon) * miles_per_deg_lon

n_vertical = ceil(lat_span_miles / cell_size)
n_horizontal = ceil(lon_span_miles / cell_size)

lat_bins = np.linspace(min_lat, max_lat, n_vertical + 1)
lon_bins = np.linspace(min_lon, max_lon, n_horizontal + 1)

data['lat_bin'] = pd.cut(data['Latitude'], bins=lat_bins, labels=False, include_lowest=True)
data['lon_bin'] = pd.cut(data['Longitude'], bins=lon_bins, labels=False, include_lowest=True)

# Prepare for Outlier Filtering 
data['Collection Date'] = pd.to_datetime(data['Collection Date'])
data['period'] = data['Collection Date'].dt.to_period('Q')  # quarterly
data['cell_id'] = data['lat_bin'].astype(str) + "_" + data['lon_bin'].astype(str)
data['Mosquito_Count_Log'] = np.log1p(data['total_mosquito'])  # log(x + 1)

# Define Rule: Ends in Two or More Zeros
def ends_in_two_or_more_zeros(x):
    return x >= 100 and str(int(x)).endswith('00')

# Outlier Removal Function with 5×IQR and "00" rule 
def remove_outliers(group):
    q1 = group['Mosquito_Count_Log'].quantile(0.25)
    q3 = group['Mosquito_Count_Log'].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + 5 * iqr  # less strict threshold
    filtered = group[group['Mosquito_Count_Log'] <= upper_bound]
    filtered = filtered[~filtered['total_mosquito'].apply(ends_in_two_or_more_zeros)]
    return filtered

# Apply Filtering 
original_count = len(data)
cleaned_data = data.groupby(['cell_id', 'period'], group_keys=False).apply(remove_outliers)
cleaned_count = len(cleaned_data)
removed_count = original_count - cleaned_count

# Results
print(f"Original datapoints: {original_count}")
print(f"Datapoints after outlier removal: {cleaned_count}")
print(f"Outliers removed: {removed_count}")

# === 8. Save Cleaned Data ===
output_path = r"C:\Users\mshahba4\Desktop\Mosquito Project\Cleaned_Mosquito_Data.csv"
cleaned_data.to_csv(output_path, index=False)


## Temporal Analysis
In the temporal analysis, we investigate how mosquito-related indicators change over time across the study period. Temporal trends are examined on two time scales: yearly (2013–2018, 2022) and monthly (aggregating across all years from 2013 to 2022). Data are grouped according to the relevant time period, and various measures such as record counts, normalized mosquito abundance, species proportions, positive case rates, and dominant diseases are computed. This part of the analysis helps identify long-term trends and seasonal patterns, which are important for understanding mosquito population dynamics and disease outbreaks.

### Step 1: Temporal Distribution of Records
In this step, we calculate the number of mosquito collection records across years and months. For the yearly plot, only records from 2013 to 2018 and 2022 are included, while for the monthly plot, all available years are aggregated. The number of records is determined by counting collection events grouped by year and month. This measure provides insight into variations in sampling effort over time, which is important for interpreting subsequent abundance and disease data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import numpy as np


data = cleaned_data.copy()


# Extract Year and Month
data['Collection Date'] = pd.to_datetime(data['Collection Date'])
data['Year'] = data['Collection Date'].dt.year
data['Month'] = data['Collection Date'].dt.month

# Yearly Plot (Only 2013-2018 and 2022)
years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
yearly_data = data[data['Year'].isin(years_of_interest)]
yearly_counts = yearly_data.groupby('Year').size().reset_index(name='Number of Records')

plt.figure(figsize=(10,6))
sns.barplot(x='Year', y='Number of Records', data=yearly_counts, color='blue')
plt.title('Number of Mosquito Collection Records Per Year (Excluding 2019-2021)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Records', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

# Monthly Plot (All years included)
monthly_counts = data.groupby('Month').size().reindex(range(1,13)).reset_index(name='Number of Records')
monthly_counts['Month Name'] = monthly_counts['Month'].apply(lambda x: calendar.month_name[x])

plt.figure(figsize=(12,6))
sns.barplot(x='Month Name', y='Number of Records', data=monthly_counts, color='red')
plt.title('Number of Mosquito Collection Records Per Month (All Years)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Number of Records', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


### Step 2: Temporal Mosquito Abundance (Normalized)
In this step, we calculate the normalized mosquito abundance over time by computing the average number of mosquitoes collected per sampling event. For each year and each month, the total number of mosquitoes is divided by the number of sampling records. This is done by grouping the data by year or month and aggregating the total mosquito counts and sample counts. This normalization corrects for differences in sampling effort across periods and allows direct comparisons of mosquito abundance trends, independent of how many traps were deployed.

In [ ]:
# --- Temporal Analysis of Normalized Mosquito Abundance ---

# Yearly Plot (Only 2013-2018 and 2022)
years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
yearly_data = data[data['Year'].isin(years_of_interest)]

yearly_abundance = yearly_data.groupby('Year').agg(
    total_mosquitoes=('total_mosquito', 'sum'),
    total_samples=('total_mosquito', 'count')
).reset_index()

yearly_abundance['Normalized Abundance'] = yearly_abundance['total_mosquitoes'] / yearly_abundance['total_samples']

plt.figure(figsize=(10,6))
sns.barplot(x='Year', y='Normalized Abundance', data=yearly_abundance, color='blue')
plt.title('Normalized Mosquito Abundance Per Year (Excluding 2019-2021)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Average Number of Mosquitoes per Sample', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

# Monthly Plot (All years included)
monthly_abundance = data.groupby('Month').agg(
    total_mosquitoes=('total_mosquito', 'sum'),
    total_samples=('total_mosquito', 'count')
).reindex(range(1,13)).reset_index()

monthly_abundance['Normalized Abundance'] = monthly_abundance['total_mosquitoes'] / monthly_abundance['total_samples']
monthly_abundance['Month Name'] = monthly_abundance['Month'].apply(lambda x: calendar.month_name[x])

plt.figure(figsize=(12,6))
sns.barplot(x='Month Name', y='Normalized Abundance', data=monthly_abundance, color='red')
plt.title('Normalized Mosquito Abundance Per Month (All Years)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average Number of Mosquitoes per Sample', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()


### Step 3: Temporal Species Composition
In this step, we evaluate the temporal composition of mosquito species by determining the proportion of each species among all identified mosquitoes collected over time. Only samples with a clearly identified species were included in the analysis (only rows where the Species column is in our predefined species list are included). This allows us to visualize how species dominance shifts between years and seasons, offering clues about ecological dynamics and species-specific responses to environmental changes.

In [ ]:
# --- Temporal Analysis of Species Composition ---

import matplotlib.pyplot as plt

# Filter valid species
species_list = ['Cx tarsalis', 'Cx quinquefasciatus', 'Ae aegypti', 'Ae vexans',
                'Ps columbiae', 'Ps species', 'Cs incidens', 'Cs species',
                'An species', 'Bird']

data_species = data[data['Species'].isin(species_list)].copy()

species_colors = sns.color_palette("tab10", n_colors=len(species_list))
species_color_dict = {species: color for species, color in zip(species_list, species_colors)}

# Yearly Composition (only 2013-2018, 2022)
years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
yearly_species = data_species[data_species['Year'].isin(years_of_interest)]

yearly_counts = yearly_species.groupby(['Year', 'Species']).size().reset_index(name='count')
yearly_total = yearly_counts.groupby('Year')['count'].sum().reset_index(name='total_count')
yearly_counts = yearly_counts.merge(yearly_total, on='Year')
yearly_counts['proportion'] = yearly_counts['count'] / yearly_counts['total_count']

yearly_pivot = yearly_counts.pivot(index='Year', columns='Species', values='proportion').fillna(0)
yearly_pivot = yearly_pivot[species_list]  # ensure correct species order

# Plot yearly species composition
yearly_pivot.plot(kind='bar', stacked=True, color=[species_color_dict[sp] for sp in species_list], figsize=(12,7))
plt.title('Species Composition by Year (Excluding 2019-2021)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Proportion of Identified Species', fontsize=12)
plt.legend(title='Species', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.yticks(np.linspace(0, 1, 6))
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

# Monthly Composition (all years)
monthly_species = data_species.copy()

monthly_counts = monthly_species.groupby(['Month', 'Species']).size().reset_index(name='count')
monthly_total = monthly_counts.groupby('Month')['count'].sum().reset_index(name='total_count')
monthly_counts = monthly_counts.merge(monthly_total, on='Month')
monthly_counts['proportion'] = monthly_counts['count'] / monthly_counts['total_count']

monthly_pivot = monthly_counts.pivot(index='Month', columns='Species', values='proportion').fillna(0)
monthly_pivot = monthly_pivot.reindex(range(1,13))  # ensure months are in order
monthly_pivot = monthly_pivot[species_list]  # ensure species order

# Plot monthly species composition
monthly_pivot.plot(kind='bar', stacked=True, color=[species_color_dict[sp] for sp in species_list], figsize=(14,7))
plt.title('Species Composition by Month (All Years)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Proportion of Identified Species', fontsize=12)
plt.legend(title='Species', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(ticks=np.arange(0,12), labels=[calendar.month_abbr[i+1] for i in range(12)], rotation=0)
plt.yticks(np.linspace(0, 1, 6))
plt.ylim(0, 1)
plt.tight_layout()
plt.show()


### Step 4: Temporal Disease Positivity Rate (In Valid Tests)
In this step, we assess the positive case rate over time by analyzing only valid disease tests, where a specific disease screening was conducted and recorded. We group the data by year and by month, counting total valid tests and positive results in each group. For each year and month, the proportion of positive results (Result = 1) out of all valid tests is calculated. This metric highlights how the likelihood of detecting a disease-positive mosquito fluctuates over time, providing insights into periods of elevated pathogen risk.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

# --- Temporal Analysis of Positive Case Rate (Corrected) ---

# Filter valid disease tests (Disease ≠ 'None')
valid_data = data[data['Disease'].notna() & (data['Disease'] != 'None')].copy()

# Yearly Plot (Only 2013–2018 and 2022)
years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
yearly_valid = valid_data[valid_data['Year'].isin(years_of_interest)]

yearly_total_tests = yearly_valid.groupby('Year').size().reset_index(name='total_tests')
yearly_positive_tests = yearly_valid[yearly_valid['Result'] == 1].groupby('Year').size().reset_index(name='positive_tests')

yearly_merge = pd.merge(yearly_total_tests, yearly_positive_tests, on='Year', how='left').fillna(0)
yearly_merge['Positive Rate'] = (yearly_merge['positive_tests'] / yearly_merge['total_tests']) * 100  # Convert to percent

yearly_max_rate = yearly_merge['Positive Rate'].max()

plt.figure(figsize=(10,6))
sns.barplot(x='Year', y='Positive Rate', data=yearly_merge, color='blue')
plt.title('Positive Disease Case Rate Per Year (Excluding 2019–2021)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Positive Rate (%)', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.ylim(0, yearly_max_rate * 1.1)
plt.tight_layout()
plt.show()

# Monthly Plot (All years included)
monthly_total_tests = valid_data.groupby('Month').size().reset_index(name='total_tests')
monthly_positive_tests = valid_data[valid_data['Result'] == 1].groupby('Month').size().reset_index(name='positive_tests')

monthly_merge = pd.merge(monthly_total_tests, monthly_positive_tests, on='Month', how='left').fillna(0)
monthly_merge['Positive Rate'] = (monthly_merge['positive_tests'] / monthly_merge['total_tests']) * 100  # Convert to percent
monthly_merge = monthly_merge.sort_values('Month')  # Ensure months are in order
monthly_merge['Month Name'] = monthly_merge['Month'].apply(lambda x: calendar.month_abbr[x])

monthly_max_rate = monthly_merge['Positive Rate'].max()

plt.figure(figsize=(12,6))
sns.barplot(x='Month Name', y='Positive Rate', data=monthly_merge, color='red')
plt.title('Positive Disease Case Rate Per Month (All Years)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Positive Rate (%)', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.ylim(0, monthly_max_rate * 1.1)
plt.tight_layout()
plt.show()


### Step 5: Temporal Disease Distribution (Among Positives)
In this step, we examine the temporal distribution of detected diseases among positive mosquito samples. Only samples that tested positive are included, and the proportion of positive cases attributable to each disease is calculated for each year and month. The number of positive cases for each disease is counted per year and per month, and the total number of positive samples across all diseases is calculated for each period. Then, the proportion of each disease relative to the total positive cases is computed. This helps to understand which pathogens dominate at different times, offering information for disease surveillance.

In [ ]:
# --- Temporal Analysis of Disease Distribution (Among Positives) ---

# Filter positive disease tests
positive_data = data[(data['Disease'].notna()) & (data['Disease'] != 'None') & (data['Result'] == 1)].copy()

disease_list = sorted(positive_data['Disease'].unique())
disease_colors = sns.color_palette("tab10", n_colors=len(disease_list))
disease_color_dict = {disease: color for disease, color in zip(disease_list, disease_colors)}

# Yearly Composition
years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
yearly_positive = positive_data[positive_data['Year'].isin(years_of_interest)]

yearly_counts = yearly_positive.groupby(['Year', 'Disease']).size().reset_index(name='count')
yearly_total = yearly_counts.groupby('Year')['count'].sum().reset_index(name='total_count')
yearly_counts = yearly_counts.merge(yearly_total, on='Year')
yearly_counts['proportion'] = yearly_counts['count'] / yearly_counts['total_count']

yearly_pivot = yearly_counts.pivot(index='Year', columns='Disease', values='proportion').fillna(0)
yearly_pivot = yearly_pivot[disease_list]  # Ensure consistent disease order

# Plot yearly disease composition
yearly_pivot.plot(kind='bar', stacked=True, color=[disease_color_dict[d] for d in disease_list], figsize=(12,7))
plt.title('Disease Composition Among Positive Samples by Year (Excluding 2019–2021)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Proportion of Positive Cases', fontsize=12)
plt.legend(title='Disease', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.yticks(np.linspace(0, 1, 6))
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

# Monthly Composition (All years)
monthly_positive = positive_data.copy()

monthly_counts = monthly_positive.groupby(['Month', 'Disease']).size().reset_index(name='count')
monthly_total = monthly_counts.groupby('Month')['count'].sum().reset_index(name='total_count')
monthly_counts = monthly_counts.merge(monthly_total, on='Month')
monthly_counts['proportion'] = monthly_counts['count'] / monthly_counts['total_count']

monthly_pivot = monthly_counts.pivot(index='Month', columns='Disease', values='proportion').fillna(0)
monthly_pivot = monthly_pivot.reindex(range(1,13))  # Ensure months ordered
monthly_pivot = monthly_pivot[disease_list]  # Ensure consistent disease order

# Plot monthly disease composition
monthly_pivot.plot(kind='bar', stacked=True, color=[disease_color_dict[d] for d in disease_list], figsize=(14,7))
plt.title('Disease Composition Among Positive Samples by Month (All Years)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Proportion of Positive Cases', fontsize=12)
plt.legend(title='Disease', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(ticks=np.arange(0,12), labels=[calendar.month_abbr[i+1] for i in range(12)], rotation=0)
plt.yticks(np.linspace(0, 1, 6))
plt.ylim(0, 1)
plt.tight_layout()
plt.show()


## Spatial Analysis
In the spatial analysis, we explore how mosquito-related indicators vary geographically across the study area. The region is divided into a uniform grid with cells measuring 1 mile by 1 mile, based on actual geographic coordinates.  Indicators are computed within each grid cell and shown in three plots: total data, yearly subsets, and monthly subsets. Users can select individual years and months in the second and third plots, respectively, to explore temporal patterns. This spatially explicit approach highlights geographic hotspots of mosquito abundance, species distributions, positive disease rates, and disease distribution.

### step 1: Spatial Distribution of Records
In this step, we calculate the number of mosquito collection records in each spatial grid cell. Cells with no records are left blank, while others are colored based on the number of sampling events. This mapping of sampling effort reveals areas with intense trapping activity versus regions with limited surveillance, which is important for interpreting later abundance and disease results. With mouse hover, the number of samples is shown on each cell.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from folium import FeatureGroup, LayerControl, TileLayer
from shapely.geometry import box
from branca.colormap import linear
from IPython.display import display
import calendar
import warnings
from math import cos, radians, ceil
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Setup Grid 
cell_size = 1.0  # miles

min_lon, max_lon = data['Longitude'].min(), data['Longitude'].max()
min_lat, max_lat = data['Latitude'].min(), data['Latitude'].max()

miles_per_deg_lat = 69.0
center_lat = (min_lat + max_lat) / 2
miles_per_deg_lon = 69.0 * cos(radians(center_lat))

lat_span_miles = (max_lat - min_lat) * miles_per_deg_lat
lon_span_miles = (max_lon - min_lon) * miles_per_deg_lon
n_vertical = ceil(lat_span_miles / cell_size)
n_horizontal = ceil(lon_span_miles / cell_size)

lat_bins = np.linspace(min_lat, max_lat, n_vertical + 1)
lon_bins = np.linspace(min_lon, max_lon, n_horizontal + 1)

data['lat_bin'] = pd.cut(data['Latitude'], bins=lat_bins, labels=False, include_lowest=True)
data['lon_bin'] = pd.cut(data['Longitude'], bins=lon_bins, labels=False, include_lowest=True)

data['Collection Date'] = pd.to_datetime(data['Collection Date'], errors='coerce')
data['Year'] = data['Collection Date'].dt.year
data['Month'] = data['Collection Date'].dt.month

center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# Geometry Helper 
def create_geometries(df):
    df['lat_min'] = df['lat_bin'].apply(lambda x: lat_bins[x])
    df['lat_max'] = df['lat_bin'].apply(lambda x: lat_bins[x + 1])
    df['lon_min'] = df['lon_bin'].apply(lambda x: lon_bins[x])
    df['lon_max'] = df['lon_bin'].apply(lambda x: lon_bins[x + 1])
    df['geometry'] = df.apply(lambda row: box(row['lon_min'], row['lat_min'], row['lon_max'], row['lat_max']), axis=1)
    return gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# TOTAL MAP 
map_total = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')
total_grid = data.groupby(['lat_bin', 'lon_bin']).size().reset_index(name='count')
gdf_total = create_geometries(total_grid)
colormap_total = linear.YlOrRd_09.scale(gdf_total['count'].min(), gdf_total['count'].max())

for _, row in gdf_total.iterrows():
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, count=row['count']: {
            'fillColor': colormap_total(count),
            'color': 'black',
            'weight': 0.3,
            'fillOpacity': 0.6
        },
        tooltip=folium.Tooltip(f"Samples: {row['count']}")
    ).add_to(map_total)

colormap_total.caption = 'Total Mosquito Collection Samples'
colormap_total.add_to(map_total)

# YEARLY MAP 
map_year = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_year)

years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
yearly_grid = data[data['Year'].isin(years_of_interest)].groupby(['Year', 'lat_bin', 'lon_bin']).size().reset_index(name='count')
vmin_year = yearly_grid['count'].min()
vmax_year = yearly_grid['count'].max()
colormap_year = linear.YlOrRd_09.scale(vmin_year, vmax_year)
colormap_year.caption = 'Yearly Mosquito Collection Samples'

for year in years_of_interest:
    df_year = yearly_grid[yearly_grid['Year'] == year].copy()
    gdf_year = create_geometries(df_year)
    fg_year = FeatureGroup(name=f"Year {year}", overlay=False, show=(year == 2022))

    for _, row in gdf_year.iterrows():
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, count=row['count']: {
                'fillColor': colormap_year(count),
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.6
            },
            tooltip=folium.Tooltip(f"Year {year}<br>Samples: {row['count']}")
        ).add_to(fg_year)

    fg_year.add_to(map_year)

colormap_year.add_to(map_year)
LayerControl(collapsed=False).add_to(map_year)

# MONTHLY MAP
map_month = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_month)

month_name_mapping = {i: calendar.month_name[i] for i in range(1, 13)}
monthly_grid = data.groupby(['Month', 'lat_bin', 'lon_bin']).size().reset_index(name='count')
vmin_month = monthly_grid['count'].min()
vmax_month = monthly_grid['count'].max()
colormap_month = linear.YlOrRd_09.scale(vmin_month, vmax_month)
colormap_month.caption = 'Monthly Mosquito Collection Samples'

for month in range(1, 13):
    df_month = monthly_grid[monthly_grid['Month'] == month].copy()
    gdf_month = create_geometries(df_month)
    fg_month = FeatureGroup(name=calendar.month_name[month], overlay=False, show=(month == 7))

    for _, row in gdf_month.iterrows():
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, count=row['count']: {
                'fillColor': colormap_month(count),
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.6
            },
            tooltip=folium.Tooltip(f"{calendar.month_name[month]}<br>Samples: {row['count']}")
        ).add_to(fg_month)

    fg_month.add_to(map_month)

colormap_month.add_to(map_month)
LayerControl(collapsed=False).add_to(map_month)

# Display all 3 Maps
display(map_total)
display(map_year)
display(map_month)


### Step 2: Spatial Mosquito Abundance (Normalized)
In this step, we determine the normalized mosquito abundance per spatial cell by calculating the average number of mosquitoes collected per sampling event within each cell. For each cell, the total mosquito count is divided by the number of samples taken there. This highlights true mosquito density patterns across the landscape and helps to identify density hotspots. With mouse hover, each cell displays the total mosquito count, number of samples, and the average mosquitoes per sample.

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium import FeatureGroup, TileLayer, LayerControl
from shapely.geometry import box
from branca.colormap import linear
import calendar

# Convert date and extract year, month
data['Collection Date'] = pd.to_datetime(data['Collection Date'], errors='coerce')
data['Year'] = data['Collection Date'].dt.year
data['Month'] = data['Collection Date'].dt.month

# Calculate total mosquito count
data['total_mosquito'] = data['Males'] + data['Females']

# Compute center coordinates
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# Geometry Helper
def create_geometries(df):
    df['lat_min'] = df['lat_bin'].apply(lambda x: lat_bins[x])
    df['lat_max'] = df['lat_bin'].apply(lambda x: lat_bins[x + 1])
    df['lon_min'] = df['lon_bin'].apply(lambda x: lon_bins[x])
    df['lon_max'] = df['lon_bin'].apply(lambda x: lon_bins[x + 1])
    df['geometry'] = df.apply(lambda row: box(row['lon_min'], row['lat_min'], row['lon_max'], row['lat_max']), axis=1)
    return gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# -------------------- TOTAL AVERAGE ABUNDANCE MAP --------------------
map_total = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')

total_group = data.groupby(['lat_bin', 'lon_bin']).agg(
    total_mosquito_sum=('total_mosquito', 'sum'),
    sample_count=('total_mosquito', 'count')
).reset_index()
total_group['normalized_abundance'] = total_group['total_mosquito_sum'] / total_group['sample_count']

gdf_total = create_geometries(total_group)

colormap_total = linear.YlOrRd_09.scale(
    gdf_total['normalized_abundance'].min(),
    gdf_total['normalized_abundance'].max()
)
colormap_total.caption = 'Avg. Mosquito Count per Sample (Total)'

for _, row in gdf_total.iterrows():
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, value=row['normalized_abundance']: {
            'fillColor': colormap_total(value),
            'color': 'black',
            'weight': 0.3,
            'fillOpacity': 0.6
        },
        tooltip=folium.Tooltip(
            f"Total Mosquito: {row['total_mosquito_sum']}<br>"
            f"Samples: {row['sample_count']}<br>"
            f"Avg Mosquito per Sample: {row['normalized_abundance']:.2f}"
        )
    ).add_to(map_total)

colormap_total.add_to(map_total)

# -------------------- YEARLY AVERAGE ABUNDANCE MAP --------------------
map_year = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_year)

years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]

yearly_group = data[data['Year'].isin(years_of_interest)].groupby(['Year', 'lat_bin', 'lon_bin']).agg(
    total_mosquito_sum=('total_mosquito', 'sum'),
    sample_count=('total_mosquito', 'count')
).reset_index()
yearly_group['normalized_abundance'] = yearly_group['total_mosquito_sum'] / yearly_group['sample_count']

colormap_year = linear.YlOrRd_09.scale(
    yearly_group['normalized_abundance'].min(),
    yearly_group['normalized_abundance'].max()
)
colormap_year.caption = 'Avg. Mosquito Count per Sample (Yearly)'

for year in years_of_interest:
    df_year = yearly_group[yearly_group['Year'] == year].copy()
    gdf_year = create_geometries(df_year)
    fg_year = FeatureGroup(name=f"Year {year}", overlay=False, show=(year == 2022))

    for _, row in gdf_year.iterrows():
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, value=row['normalized_abundance']: {
                'fillColor': colormap_year(value),
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.6
            },
            tooltip=folium.Tooltip(
                f"Year {year}<br>"
                f"Total Mosquito: {row['total_mosquito_sum']}<br>"
                f"Samples: {row['sample_count']}<br>"
                f"Avg Mosquito per Sample: {row['normalized_abundance']:.2f}"
            )
        ).add_to(fg_year)

    fg_year.add_to(map_year)

colormap_year.add_to(map_year)
LayerControl(collapsed=False).add_to(map_year)

# -------------------- MONTHLY AVERAGE ABUNDANCE MAP --------------------
map_month = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_month)

monthly_group = data.groupby(['Month', 'lat_bin', 'lon_bin']).agg(
    total_mosquito_sum=('total_mosquito', 'sum'),
    sample_count=('total_mosquito', 'count')
).reset_index()
monthly_group['normalized_abundance'] = monthly_group['total_mosquito_sum'] / monthly_group['sample_count']

colormap_month = linear.YlOrRd_09.scale(
    monthly_group['normalized_abundance'].min(),
    monthly_group['normalized_abundance'].max()
)
colormap_month.caption = 'Avg. Mosquito Count per Sample (Monthly)'

for month in range(1, 13):
    df_month = monthly_group[monthly_group['Month'] == month].copy()
    gdf_month = create_geometries(df_month)
    fg_month = FeatureGroup(name=calendar.month_name[month], overlay=False, show=(month == 7))

    for _, row in gdf_month.iterrows():
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, value=row['normalized_abundance']: {
                'fillColor': colormap_month(value),
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.6
            },
            tooltip=folium.Tooltip(
                f"{calendar.month_name[month]}<br>"
                f"Total Mosquito: {row['total_mosquito_sum']}<br>"
                f"Samples: {row['sample_count']}<br>"
                f"Avg Mosquito per Sample: {row['normalized_abundance']:.2f}"
            )
        ).add_to(fg_month)

    fg_month.add_to(map_month)

colormap_month.add_to(map_month)
LayerControl(collapsed=False).add_to(map_month)

# -------------------- DISPLAY ALL MAPS --------------------
display(map_total)
display(map_year)
display(map_month)


### step 3: Spatial Species Composition (Dominant Species)
In this step, we identify the dominant mosquito species in each spatial grid cell. The species with the highest number of collected mosquitoes is assigned as the dominant species for that cell. This analysis shows spatial shifts in species dominance across time periods and locations, which can indicate changes in habitat suitability or vector control effectiveness. This analysis is performed only among the identified species, excluding all others. With mouse hover, each cell displays the percentage breakdown of all species found in that location.

In [ ]:
data['Collection Date'] = pd.to_datetime(data['Collection Date'], errors='coerce')
data['Year'] = data['Collection Date'].dt.year
data['Month'] = data['Collection Date'].dt.month

center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# Species List & Colors 
species_list = ['Cx tarsalis', 'Cx quinquefasciatus', 'Ae aegypti', 'Ae vexans',
                'Ps columbiae', 'Ps species', 'Cs incidens', 'Cs species',
                'An species', 'Bird']

species_color_dict = {
    'Cx tarsalis': '#1f78b4',
    'Cx quinquefasciatus': '#ff7f00',
    'Ae aegypti': '#33a02c',
    'Ae vexans': '#e31a1c',
    'Ps columbiae': '#6a3d9a',
    'Ps species': '#b15928',
    'Cs incidens': '#cab2d6',
    'Cs species': '#666666',
    'An species': '#b2df8a',
    'Bird': '#00bfc4'
}

# Geometry Helper
def create_geometries(df):
    df['lat_min'] = df['lat_bin'].apply(lambda x: lat_bins[x])
    df['lat_max'] = df['lat_bin'].apply(lambda x: lat_bins[x + 1])
    df['lon_min'] = df['lon_bin'].apply(lambda x: lon_bins[x])
    df['lon_max'] = df['lon_bin'].apply(lambda x: lon_bins[x + 1])
    df['geometry'] = df.apply(lambda row: box(row['lon_min'], row['lat_min'], row['lon_max'], row['lat_max']), axis=1)
    return gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# Total Dominant Species Map 
data_species = data[data['Species'].isin(species_list)].copy()

map_total = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')
total_group = data_species.groupby(['lat_bin', 'lon_bin', 'Species']).size().reset_index(name='count')
cell_total = total_group.groupby(['lat_bin', 'lon_bin'])['count'].sum().reset_index(name='total_count')
total_group = total_group.merge(cell_total, on=['lat_bin', 'lon_bin'])
total_group['percent'] = 100 * total_group['count'] / total_group['total_count']
total_group['percent'] = total_group['percent'].round(1)
tooltip_dict = (
    total_group.sort_values(['lat_bin', 'lon_bin', 'percent'], ascending=[True, True, False])
    .groupby(['lat_bin', 'lon_bin'])
    .apply(lambda g: "<br>".join([f"{row['Species']}: {row['percent']}%" for _, row in g.iterrows()]))
    .reset_index(name='tooltip')
)
dominant_total = total_group.groupby(['lat_bin', 'lon_bin'], group_keys=False).apply(lambda x: x.loc[x['count'].idxmax()]).reset_index(drop=True)
dominant_total = dominant_total.merge(tooltip_dict, on=['lat_bin', 'lon_bin'])
gdf_total = create_geometries(dominant_total)

for _, row in gdf_total.iterrows():
    species = row['Species']
    tooltip_text = row['tooltip']
    color = species_color_dict.get(species, "#cccccc")
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, clr=color: {
            'fillColor': clr,
            'color': 'black',
            'weight': 0.3,
            'fillOpacity': 0.7
        },
        tooltip=folium.Tooltip(tooltip_text)
    ).add_to(map_total)

# Yearly Map 
map_year = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_year)

years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
yearly_group = data_species[data_species['Year'].isin(years_of_interest)].groupby(['Year', 'lat_bin', 'lon_bin', 'Species']).size().reset_index(name='count')
cell_total_year = yearly_group.groupby(['Year', 'lat_bin', 'lon_bin'])['count'].sum().reset_index(name='total_count')
yearly_group = yearly_group.merge(cell_total_year, on=['Year', 'lat_bin', 'lon_bin'])
yearly_group['percent'] = 100 * yearly_group['count'] / yearly_group['total_count']
yearly_group['percent'] = yearly_group['percent'].round(1)
tooltip_dict_year = (
    yearly_group.sort_values(['Year', 'lat_bin', 'lon_bin', 'percent'], ascending=[True, True, True, False])
    .groupby(['Year', 'lat_bin', 'lon_bin'])
    .apply(lambda g: "<br>".join([f"{row['Species']}: {row['percent']}%" for _, row in g.iterrows()]))
    .reset_index(name='tooltip')
)
dominant_yearly = yearly_group.groupby(['Year', 'lat_bin', 'lon_bin'], group_keys=False).apply(lambda x: x.loc[x['count'].idxmax()]).reset_index(drop=True)
dominant_yearly = dominant_yearly.merge(tooltip_dict_year, on=['Year', 'lat_bin', 'lon_bin'])

for year in years_of_interest:
    df_year = dominant_yearly[dominant_yearly['Year'] == year].copy()
    gdf_year = create_geometries(df_year)
    fg = FeatureGroup(name=f"Year {year}", overlay=False, show=(year == 2022))
    
    for _, row in gdf_year.iterrows():
        species = row['Species']
        color = species_color_dict.get(species, "#cccccc")
        tooltip_text = row['tooltip']
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, clr=color: {
                'fillColor': clr,
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.7
            },
            tooltip=folium.Tooltip(f"Year {year}<br>{tooltip_text}")
        ).add_to(fg)
    
    fg.add_to(map_year)

# Monthly Map 
map_month = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_month)

monthly_group = data_species.groupby(['Month', 'lat_bin', 'lon_bin', 'Species']).size().reset_index(name='count')
cell_total_month = monthly_group.groupby(['Month', 'lat_bin', 'lon_bin'])['count'].sum().reset_index(name='total_count')
monthly_group = monthly_group.merge(cell_total_month, on=['Month', 'lat_bin', 'lon_bin'])
monthly_group['percent'] = 100 * monthly_group['count'] / monthly_group['total_count']
monthly_group['percent'] = monthly_group['percent'].round(1)
tooltip_dict_month = (
    monthly_group.sort_values(['Month', 'lat_bin', 'lon_bin', 'percent'], ascending=[True, True, True, False])
    .groupby(['Month', 'lat_bin', 'lon_bin'])
    .apply(lambda g: "<br>".join([f"{row['Species']}: {row['percent']}%" for _, row in g.iterrows()]))
    .reset_index(name='tooltip')
)
dominant_monthly = monthly_group.groupby(['Month', 'lat_bin', 'lon_bin'], group_keys=False).apply(lambda x: x.loc[x['count'].idxmax()]).reset_index(drop=True)
dominant_monthly = dominant_monthly.merge(tooltip_dict_month, on=['Month', 'lat_bin', 'lon_bin'])

for month in range(1, 13):
    df_month = dominant_monthly[dominant_monthly['Month'] == month].copy()
    gdf_month = create_geometries(df_month)
    fg = FeatureGroup(name=calendar.month_name[month], overlay=False, show=(month == 7))

    for _, row in gdf_month.iterrows():
        species = row['Species']
        color = species_color_dict.get(species, "#cccccc")
        tooltip_text = row['tooltip']
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, clr=color: {
                'fillColor': clr,
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.7
            },
            tooltip=folium.Tooltip(f"{calendar.month_name[month]}<br>{tooltip_text}")
        ).add_to(fg)
    
    fg.add_to(map_month)

# Styled Legend & Display 
legend_html = """
<div style='
    position: fixed;
    bottom: 40px;
    left: 10px;
    width: 220px;
    z-index: 9999;
    font-size: 14px;
    background-color: white;
    padding: 10px;
    border: 1px solid black;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
'>
<b>Dominant Species</b><br>
""" + \
"".join([
    f"<i style='background:{col};width:18px;height:12px;display:inline-block;margin-right:5px'></i>{sp}<br>"
    for sp, col in species_color_dict.items()
]) + \
"</div>"

map_total.get_root().html.add_child(folium.Element(legend_html))
map_year.get_root().html.add_child(folium.Element(legend_html))
map_month.get_root().html.add_child(folium.Element(legend_html))

LayerControl(collapsed=False).add_to(map_year)
LayerControl(collapsed=False).add_to(map_month)

# Display All Maps 
display(map_total)
display(map_year)
display(map_month)

### Step 4: Spatial Disease Positivity Rate (In Valid Tests)
In this step, we calculate the spatial positive disease case rate by determining the proportion of positive disease tests among all valid tests within each grid cell. We include only valid tests where the Disease field is not labeled as 'None', ensuring that only properly tested samples are analyzed. Also, only cells with at least five valid tests are included to ensure statistical reliability. This highlights spatial hotspots where disease-positive mosquitoes are more detected, guiding targeted interventions. With mouse hover, each cell displays the positive rate, number of valid tests, and total positive cases.

In [ ]:
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from shapely.geometry import box
from branca.colormap import linear
from folium import FeatureGroup, LayerControl, TileLayer
import calendar
from IPython.display import display

# Data Preprocessing
data['Collection Date'] = pd.to_datetime(data['Collection Date'], errors='coerce')
data['Year'] = data['Collection Date'].dt.year
data['Month'] = data['Collection Date'].dt.month

data['Valid Test'] = data['Disease'].notna() & (data['Disease'] != 'None')
data['Positive Test'] = data['Result'] == 1

center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# Geometry Helper
def create_geometries(df):
    df['lat_min'] = df['lat_bin'].apply(lambda x: lat_bins[x])
    df['lat_max'] = df['lat_bin'].apply(lambda x: lat_bins[x + 1])
    df['lon_min'] = df['lon_bin'].apply(lambda x: lon_bins[x])
    df['lon_max'] = df['lon_bin'].apply(lambda x: lon_bins[x + 1])
    df['geometry'] = df.apply(lambda row: box(row['lon_min'], row['lat_min'], row['lon_max'], row['lat_max']), axis=1)
    return gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# TOTAL POSITIVE RATE MAP
valid_total = data[data['Valid Test']]
total_group = valid_total.groupby(['lat_bin', 'lon_bin']).agg(
    positive_sum=('Positive Test', 'sum'),
    valid_count=('Positive Test', 'count')
).reset_index()
total_group['positive_rate'] = 100 * total_group['positive_sum'] / total_group['valid_count']
total_group = total_group[total_group['valid_count'] >= 5]
gdf_total = create_geometries(total_group)

map_total = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')
colormap_total = linear.YlOrRd_09.scale(0, gdf_total['positive_rate'].max())
colormap_total.caption = 'Positive Test Rate (%)'

for _, row in gdf_total.iterrows():
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, rate=row['positive_rate']: {
            'fillColor': colormap_total(rate),
            'color': 'black',
            'weight': 0.3,
            'fillOpacity': 0.7
        },
        tooltip=folium.Tooltip(
            f"Positive Rate: {row['positive_rate']:.2f}%<br>"
            f"Valid Tests: {row['valid_count']}<br>"
            f"Total Positives: {row['positive_sum']}"
        )
    ).add_to(map_total)

colormap_total.add_to(map_total)

# YEARLY POSITIVE RATE MAP 
map_year = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_year)

years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
valid_yearly = data[data['Valid Test'] & data['Year'].isin(years_of_interest)]
yearly_group = valid_yearly.groupby(['Year', 'lat_bin', 'lon_bin']).agg(
    positive_sum=('Positive Test', 'sum'),
    valid_count=('Positive Test', 'count')
).reset_index()
yearly_group['positive_rate'] = 100 * yearly_group['positive_sum'] / yearly_group['valid_count']
yearly_group = yearly_group[yearly_group['valid_count'] >= 5]

vmax_yearly = yearly_group['positive_rate'].max()
colormap_yearly = linear.YlOrRd_09.scale(0, vmax_yearly)
colormap_yearly.caption = 'Positive Test Rate (%)'

for year in years_of_interest:
    df_year = yearly_group[yearly_group['Year'] == year].copy()
    gdf_year = create_geometries(df_year)
    fg = FeatureGroup(name=f"Year {year}", overlay=False, show=(year == 2022))

    for _, row in gdf_year.iterrows():
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, rate=row['positive_rate']: {
                'fillColor': colormap_yearly(rate),
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.7
            },
            tooltip=folium.Tooltip(
                f"Year: {year}<br>"
                f"Positive Rate: {row['positive_rate']:.2f}%<br>"
                f"Valid Tests: {row['valid_count']}<br>"
                f"Total Positives: {row['positive_sum']}"
            )
        ).add_to(fg)

    fg.add_to(map_year)

colormap_yearly.add_to(map_year)
LayerControl(collapsed=False).add_to(map_year)

# MONTHLY POSITIVE RATE MAP
map_month = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_month)

months = list(range(1, 13))
month_name_mapping = {i: calendar.month_name[i] for i in months}
valid_monthly = data[data['Valid Test']]
monthly_group = valid_monthly.groupby(['Month', 'lat_bin', 'lon_bin']).agg(
    positive_sum=('Positive Test', 'sum'),
    valid_count=('Positive Test', 'count')
).reset_index()
monthly_group['positive_rate'] = 100 * monthly_group['positive_sum'] / monthly_group['valid_count']
monthly_group = monthly_group[monthly_group['valid_count'] >= 5]

vmax_month = monthly_group['positive_rate'].max()
colormap_month = linear.YlOrRd_09.scale(0, vmax_month)
colormap_month.caption = 'Positive Test Rate (%)'

for month in months:
    df_month = monthly_group[monthly_group['Month'] == month].copy()
    gdf_month = create_geometries(df_month)
    fg = FeatureGroup(name=calendar.month_name[month], overlay=False, show=(month == 7))

    for _, row in gdf_month.iterrows():
        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, rate=row['positive_rate']: {
                'fillColor': colormap_month(rate),
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.7
            },
            tooltip=folium.Tooltip(
                f"{calendar.month_name[month]}<br>"
                f"Positive Rate: {row['positive_rate']:.2f}%<br>"
                f"Valid Tests: {row['valid_count']}<br>"
                f"Total Positives: {row['positive_sum']}"
            )
        ).add_to(fg)

    fg.add_to(map_month)

colormap_month.add_to(map_month)
LayerControl(collapsed=False).add_to(map_month)

# Display Maps
display(map_total)
display(map_year)
display(map_month)


### step 5: Spatial Disease Distribution 
In this step, we explore the dominant disease detected among disease-positive mosquitoe samples in each spatial grid cell. For each cell, the disease type with the highest number of positive detections is identified. This spatial mapping of dominant pathogens helps public health teams focus their efforts on areas where specific mosquito-borne diseases are most prevalent. With mouse hover, the map displays a tooltip showing the percentage breakdown of diseases in each cell.


In [ ]:

data['Collection Date'] = pd.to_datetime(data['Collection Date'], errors='coerce')
data['Year'] = data['Collection Date'].dt.year
data['Month'] = data['Collection Date'].dt.month

positive_data = data[(data['Disease'].notna()) & (data['Disease'] != "None") & (data['Result'] == 1)].copy()
disease_list = sorted(positive_data['Disease'].unique())

custom_colors = ['blue', 'orange', 'green']
if len(disease_list) > 3:
    custom_colors += ['#cccccc'] * (len(disease_list) - 3)
disease_color_dict = {disease: custom_colors[i] for i, disease in enumerate(disease_list)}

center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# Geometry Helper
def create_geometries(df):
    df = df.dropna(subset=['lat_bin', 'lon_bin']).copy()
    df['lat_bin'] = df['lat_bin'].astype(int)
    df['lon_bin'] = df['lon_bin'].astype(int)
    df = df[(df['lat_bin'] >= 0) & (df['lat_bin'] < len(lat_bins) - 1)]
    df = df[(df['lon_bin'] >= 0) & (df['lon_bin'] < len(lon_bins) - 1)]

    lat_min = lat_bins[df['lat_bin'].values]
    lat_max = lat_bins[df['lat_bin'].values + 1]
    lon_min = lon_bins[df['lon_bin'].values]
    lon_max = lon_bins[df['lon_bin'].values + 1]

    df['lat_min'] = lat_min
    df['lat_max'] = lat_max
    df['lon_min'] = lon_min
    df['lon_max'] = lon_max

    geometries = [
        box(lon_min[i], lat_min[i], lon_max[i], lat_max[i])
        for i in range(len(df))
    ]
    return gpd.GeoDataFrame(df, geometry=geometries, crs='EPSG:4326')

# Legend Helper
def create_disease_legend_html(disease_color_dict):
    html = """
    <div style='
        position: fixed;
        bottom: 40px;
        left: 10px;
        width: 220px;
        z-index: 9999;
        font-size: 14px;
        background-color: white;
        padding: 10px;
        border: 1px solid black;
        box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    '>
    <b>Dominant Disease</b><br>
    """
    for disease, color in disease_color_dict.items():
        html += f"<i style='background:{color};width:18px;height:12px;display:inline-block;margin-right:5px'></i>{disease}<br>"
    html += "</div>"
    return html

legend_html = create_disease_legend_html(disease_color_dict)

# Function to Add GeoJson 
def format_disease_percentages(df_counts, group_keys):
    tooltips = {}
    for key, group in df_counts.groupby(group_keys):
        total = group['count'].sum()
        if total == 0:
            continue
        breakdown = group.copy()
        breakdown['percent'] = (breakdown['count'] / total * 100).round(1)
        breakdown = breakdown[breakdown['percent'] > 0]
        breakdown = breakdown.sort_values(by='percent', ascending=False)

        tooltip_text = "<b>Disease Breakdown</b><br>"
        for _, row in breakdown.iterrows():
            tooltip_text += f"{row['Disease']}: {row['percent']}%<br>"

        tooltips[key] = tooltip_text
    return tooltips

def add_layer_to_map(map_obj, gdf, tooltips, name=None, show=False, extra_label=''):
    fg = FeatureGroup(name=name, overlay=False, show=show) if name else map_obj

    for _, row in gdf.iterrows():
        key = (row.get('Year') or row.get('Month') or None, row['lat_bin'], row['lon_bin']) if 'Year' in row or 'Month' in row else (row['lat_bin'], row['lon_bin'])
        tooltip = tooltips.get(key, "")
        disease = row['Disease']
        color = disease_color_dict.get(disease, "#cccccc")

        folium.GeoJson(
            row['geometry'].__geo_interface__,
            style_function=lambda feature, clr=color: {
                'fillColor': clr,
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.7
            },
            tooltip=folium.Tooltip(f"{extra_label}{tooltip}")
        ).add_to(fg)

    if name:
        fg.add_to(map_obj)

# TOTAL Map 
map_total = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')
total_group = positive_data.groupby(['lat_bin', 'lon_bin', 'Disease']).size().reset_index(name='count')
tooltip_total = format_disease_percentages(total_group, ['lat_bin', 'lon_bin'])
idx = total_group.groupby(['lat_bin', 'lon_bin'])['count'].idxmax()
dominant_total = total_group.loc[idx].reset_index(drop=True)
gdf_total = create_geometries(dominant_total)
add_layer_to_map(map_total, gdf_total, tooltip_total)
map_total.get_root().html.add_child(folium.Element(legend_html))

# YEARLY Map
map_year = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_year)
years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018, 2022]
yearly_group = positive_data[positive_data['Year'].isin(years_of_interest)].groupby(
    ['Year', 'lat_bin', 'lon_bin', 'Disease']).size().reset_index(name='count')
tooltip_year = format_disease_percentages(yearly_group, ['Year', 'lat_bin', 'lon_bin'])
idx = yearly_group.groupby(['Year', 'lat_bin', 'lon_bin'])['count'].idxmax()
dominant_yearly = yearly_group.loc[idx].reset_index(drop=True)

for year in years_of_interest:
    df_year = dominant_yearly[dominant_yearly['Year'] == year].copy()
    gdf_year = create_geometries(df_year)
    add_layer_to_map(map_year, gdf_year, tooltip_year, name=f"Year {year}", show=(year == 2022), extra_label=f"Year: {year}<br>")

map_year.get_root().html.add_child(folium.Element(legend_html))
LayerControl(collapsed=False).add_to(map_year)

# MONTHLY Map 
map_month = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles=None)
TileLayer('OpenStreetMap', name='Base Map', control=False).add_to(map_month)
monthly_group = positive_data.groupby(['Month', 'lat_bin', 'lon_bin', 'Disease']).size().reset_index(name='count')
tooltip_month = format_disease_percentages(monthly_group, ['Month', 'lat_bin', 'lon_bin'])
idx = monthly_group.groupby(['Month', 'lat_bin', 'lon_bin'])['count'].idxmax()
dominant_monthly = monthly_group.loc[idx].reset_index(drop=True)

for month in range(1, 13):
    df_month = dominant_monthly[dominant_monthly['Month'] == month].copy()
    gdf_month = create_geometries(df_month)
    add_layer_to_map(map_month, gdf_month, tooltip_month, name=calendar.month_name[month], show=(month == 7), extra_label=f"{calendar.month_name[month]}<br>")

map_month.get_root().html.add_child(folium.Element(legend_html))
LayerControl(collapsed=False).add_to(map_month)

# Display All Maps
display(map_total)
display(map_year)
display(map_month)